### Import Libraries


---



In [ ]:
import tensorflow as tf
import numpy as np
import os
import time

### Check GPU functionality

In [ ]:
tf.config.list_physical_devices('GPU')

In [ ]:
tf.test.is_built_with_cuda()

In [ ]:
tf.test.gpu_device_name()
!nvidia-smi

### Download Light Novel Title dataset


In [ ]:
filepath = tf.keras.utils.get_file('light_novel_titles_clean_v2.txt', 'https://drive.google.com/uc?export=download&id=13ExvJcOr0l8LZD1gHCiYJn2C_pKx8v1D')

### Check data content

In [ ]:
text = open(filepath, 'rb').read().decode(encoding='utf-16')
print(f'Length of text: {len(text)} characters')

In [ ]:
# Check first 250 characters
print(text[:250])

In [ ]:
# Check amount of unique characters
vocab = sorted(set(text))
print(f'{len(vocab)} unique characters')


## Text Processing

### Text Vectorization


In [ ]:
chars_to_ids = tf.keras.layers.StringLookup(vocabulary=list(vocab), mask_token=None)
ids_to_chars = tf.keras.layers.StringLookup(vocabulary=chars_to_ids.get_vocabulary(), invert=True, mask_token=None)

def text_from_ids(ids):
  return tf.strings.reduce_join(ids_to_chars(ids), axis =-1)

### Training Examples

In [ ]:
all_ids = chars_to_ids(tf.strings.unicode_split(text,'UTF-8'))
all_ids

In [ ]:
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)

In [ ]:
for ids in ids_dataset.take(10):
  print(ids_to_chars(ids).numpy().decode('UTF-8'))

In [ ]:
seq_length = 100
examples_per_epoch = len(text)//(seq_length)+1


In [ ]:
sequences = ids_dataset.batch(seq_length+1, drop_remainder=True)
for seq in sequences.take(1):
  print(ids_to_chars(seq))

In [ ]:
for seq in sequences.take(5):
  print(text_from_ids(seq).numpy())

In [ ]:
def split_input_target(sequence):
  input_text = sequence[:-1]
  target_text = sequence[1:]
  return input_text, target_text

In [ ]:
dataset = sequences.map(split_input_target)

In [ ]:
for input_example, target_example in dataset.take(1):
  print("Input:", text_from_ids(input_example).numpy())
  print("Target:", text_from_ids(target_example).numpy())

### Training batch creation

In [ ]:
# Batch size
BATCH_SIZE = 64
# Buffer size for dataset shuffle
BUFFER_SIZE = 10000

dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))

dataset

## Build Model

In [ ]:
# Length of vocabulary in chars
vocab_size = len(vocab)

# Embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [ ]:
class MyModel(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, rnn_units):
    super().__init__(self)
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(rnn_units,
                                   return_sequences=True,
                                   return_state=True)
    self.dense = tf.keras.layers.Dense(vocab_size)
                                    
  def call(self, inputs, states=None, return_state=False, training=False):
    x = inputs
    x = self.embedding(x, training=training)
    if states is None:
      states = self.gru.get_initial_state(x)
    x, states = self.gru(x, initial_state=states, training=training)
    x = self.dense(x, training=training)

    if return_state:
      return x, states
    else:
      return x

In [ ]:
model = MyModel(
    # Be sure vocabulary size matches the 'StringLookup' layers.
    vocab_size=len(chars_to_ids.get_vocabulary()),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)


## Model test

In [ ]:
for input_example_batch, target_example_batch in dataset.take(1):
  example_batch_predictions = model(input_example_batch)
  print(example_batch_predictions.shape, "(batch_size, sequence_length, vocab_size)")

In [ ]:
model.summary()

In [ ]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples = 1)
sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy()

In [ ]:
sampled_indices

In [ ]:
# Decode untrained model prediction
print("Input:\n", text_from_ids(input_example_batch[0]).numpy())
print()
print("Next Char Predictions:\n", text_from_ids(sampled_indices).numpy())

## Model Training

### Optimizer and Loss Function

In [ ]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

In [ ]:
example_batch_mean_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, "(batch_size, sequence_length, vocab_size)")
print("Mean loss:        ", example_batch_mean_loss)


In [ ]:
tf.exp(example_batch_mean_loss).numpy()

In [ ]:
model.compile(optimizer='adam', loss=loss)

### Configure checkpoints

In [ ]:
# Checkpoint save directory
checkpoint_dir = './training_checkpoints'

# Name of checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

### Start training

In [ ]:
EPOCHS=100

In [ ]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

In [ ]:
TEMP = 0.5

In [ ]:
class OneStep(tf.keras.Model):
  def __init__(self, model, ids_to_chars, chars_to_ids, temperature=TEMP):
    super().__init__()
    self.temperature = temperature
    self.model = model
    self.ids_to_chars = ids_to_chars
    self.chars_to_ids = chars_to_ids

    # Create a mask to prevent "[UNK]" from being generated.
    skip_ids = self.chars_to_ids(['[UNK]'])[:, None]
    sparse_mask = tf.SparseTensor(
        # Put a -inf at each bad index.
        values=[-float('inf')]*len(skip_ids),
        indices=skip_ids,
        # Match the shape to the vocabulary
        dense_shape=[len(chars_to_ids.get_vocabulary())])
    self.prediction_mask = tf.sparse.to_dense(sparse_mask)

  @tf.function
  def generate_one_step(self, inputs, states=None):
    # Convert strings to token IDs.
    input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
    input_ids = self.chars_to_ids(input_chars).to_tensor()

    # Run model
    # predicted_logits.shape is [batch, char, next_char_logits]
    predicted_logits, states = self.model(inputs=input_ids, states=states,
                                          return_state=True)
    # Only use the last prediction.
    predicted_logits = predicted_logits[:, -1, :]
    predicted_logits = predicted_logits/self.temperature
    # Apply prediction mask: prevent UNK generation.
    predicted_logits = predicted_logits + self.prediction_mask

    # Sample output logits to generate token IDs.
    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
    predicted_ids = tf.squeeze(predicted_ids, axis=-1)

    # Convert token ids to characters
    predicted_chars = self.ids_to_chars(predicted_ids)

    # Return characters and model state.
    return predicted_chars, states

In [ ]:
one_step_model = OneStep(model, ids_to_chars, chars_to_ids)

In [ ]:
SEED = "Is this a light novel title?"

In [ ]:
start = time.time()
states = None
# Start String
next_char = tf.constant([SEED])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  # TODO: Check if generated title is in source text, if yes -> regenerate
  result.append(next_char)

result = tf.strings.join(result)
result = result[0].numpy().decode('utf-8')
end = time.time()
print(result)
print('\nRun time:', end - start)